# Randomized Contraction

Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on the above graph to compute the min cut.  (HINT: Note that you'll have to figure out an implementation of edge contractions.  Initially, you might want to do this naively, creating a new graph from the old every time there's an edge contraction.  But you should also think about more efficient implementations.) 

We will define an "inplace" contraction routine so that we may not overuse RAM memory

In [1]:
class Graph():
    def __init__(self, dict_of_lists):
        """
        Initialize the graph with an adjacency list.

        Arguments:
            dict_of_lists (dict): Dictionary where keys are vertices and values are lists of adjacent vertices.
        """
        self.graph = dict_of_lists
        self.v = len(dict_of_lists.keys())

    def copy(self):
        """
        Create a deep copy of the graph.

        Returns:
            Graph: A new graph instance with the same structure and edges.
        """
        l = {}
        for i, list_ in self.graph.items():
            l[i] = list_.copy()
        return Graph(l)

    def contract(self, u, v):
        """
        Merge vertex v into vertex u by contracting the edge between them.
        Updates all references of v to u and removes v from the graph.

        Arguments:
            u: The vertex to keep after the merge.
            v: The vertex to merge into u.
        """
        if v not in self.graph[u] or u not in self.graph[v]:
            return

        while v in self.graph[u]:
            self.graph[u].remove(v)
        while u in self.graph[v]:
            self.graph[v].remove(u)

        self.graph[u].extend(self.graph[v])

        for x in list(self.graph.keys()):
            if x == v:
                continue
            self.graph[x] = [u if neighbor == v else neighbor for neighbor in self.graph[x]]

        self.graph[u] = [x for x in self.graph[u] if x != u]

        del self.graph[v]

    def removeVertex(self, v):
        """
        Remove a vertex and all its associated edges from the graph.

        Arguments:
            v: The vertex to be removed.
        """
        if v not in self.graph:
            return

        del self.graph[v]
        for i in self.graph.keys():
            if v in self.graph[i]:
                self.graph[i].remove(v)

    def countEdges(self):
        """
        Count the total number of edges in the undirected graph.

        Returns:
            int: The number of edges in the graph.
        """
        count = 0
        for i in self.graph.keys():
            count += len(self.graph[i])
        return count // 2


Now we make the randomized contraction routine using the numpy random states

In [2]:
import numpy as np

def randomizedContraction(G, randomState=42):
    """
    Apply Karger's randomized contraction algorithm once to estimate the minimum cut.

    Arguments:
        G (Graph): An instance of the Graph class to be contracted.
        randomState (int): Seed for reproducibility of random choices.

    Returns:
        int: The number of edges between the two remaining supernodes, representing a cut.
    """
    # Make a copy of the graph
    g = G.copy()
    np.random.seed(randomState)

    # While there are more than 2 vertices in the graph
    while len(g.graph) > 2:
        # Choose a random edge
        u = np.random.choice(list(g.graph.keys()))
        v = np.random.choice(g.graph[u])
        # Contract the edge
        g.contract(u, v)

    c = g.countEdges()
    return c

def trailsContraction(graph, trials=10, verbose=False, prints=5):
    """
    Run multiple trials of the randomized contraction algorithm to increase the chances of finding the true minimum cut.

    Args:
        graph (Graph): The input graph on which to perform the contraction trials.
        trials (int or bool): Number of trials to run. If True, automatically compute a high-probability trial count.
        verbose (bool): Whether to print progress messages during execution.
        prints (int): Frequency of printed updates when verbose is enabled.

    Returns:
        int: The smallest cut value found across all trials.
    """
    if trials == True:
        trials = len(graph.graph.keys())
        trials = int(trials**2 * np.log(trials))
        print('Number of trials set to', trials)

    width = len(str(trials))
    minCut = float('inf')

    for i in range(trials):
        if verbose and i % prints == 0:
            print(f"Trial {i:<{width}} of {trials:<{width}} --> Current best min cut: {minCut}")
        minCut = min(minCut, randomizedContraction(graph, randomState=i))

    return minCut


We test our implementation using an small example

In [3]:
test = {
    0: [1,2],
    1: [0,2,3],
    2: [0,1,3],
    3: [1,2]
}

G_example = Graph(test)
c = trailsContraction(G_example, trials=True)
print(f"Minimum cut: {c}")

Number of trials set to 22
Minimum cut: 2


Now we apply it to the case given in the assignment

In [4]:
with open('kargerMinCut.txt', 'r') as f:
    data = f.readlines()
    g = {}
    for line in data:
        line = line.split()
        if line == '':
            continue
        # Split the line into a list of integers
        g[int(line[0])] = list(map(int, line))[1:]

G = Graph(g)
print(f'Number of vertices: {G.v}')
print(f'Number of edges: {G.countEdges()}')

c = trailsContraction(G, trials=500, verbose = True, prints = 50)
print(f"Minimum cut: {c}")

Number of vertices: 200
Number of edges: 2517
Trial 0   of 500 --> Current best min cut: inf
Trial 50  of 500 --> Current best min cut: 17
Trial 100 of 500 --> Current best min cut: 17
Trial 150 of 500 --> Current best min cut: 17
Trial 200 of 500 --> Current best min cut: 17
Trial 250 of 500 --> Current best min cut: 17
Trial 300 of 500 --> Current best min cut: 17
Trial 350 of 500 --> Current best min cut: 17
Trial 400 of 500 --> Current best min cut: 17
Trial 450 of 500 --> Current best min cut: 17
Minimum cut: 17
